# Hackathon 1 - 120 years of Olympic history

## Problem Definition

This dataset provides an opportunity to ask questions about how the Olympics have evolved over time, including questions about the participation and performance of women, different nations, and different sports and events.

* **Problem space**
    1. Using clustering to group athlete's features together, can we get some insights on which features are useful to win!
    

* **Objective and success metrics**
    1. Produce a world map of medal winners.

## Import librairies

In [113]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, export_png
from bokeh.models import GeoJSONDataSource, HoverTool, LogColorMapper, CategoricalColorMapper, LinearColorMapper, ColorBar, BasicTicker, PrintfTickFormatter, ZoomInTool, ZoomOutTool, PanTool
from bokeh.plotting import figure, show
from bokeh.palettes import Category20c, viridis, Viridis256, RdYlBu, RdBu, Spectral
import os
import imageio
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import pandas_profiling
output_notebook()

Loading BokehJS ...

## Data preparation

### Select and source data

In [24]:
# load athlete and event data
df = pd.read_csv('./datasets/athlete_events.csv')
df.shape

(271116, 15)

In [25]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [26]:
# load noc regions data
regions = pd.read_csv('./datasets/noc_regions.csv')
regions.shape

(230, 3)

In [27]:
regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [28]:
# let's merge both dataframes into df
df = pd.merge(df, regions, on='NOC', how='left')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


### Data exploration

In [11]:
profile = pandas_profiling.ProfileReport(df)
display(profile)

Number of variables,17
Number of observations,271116
Total Missing (%),13.7%
Total size in memory,37.2 MiB
Average record size in memory,144.0 B
Numeric,5
Categorical,12
Boolean,0
Date,0
Text (Unique),0
Rejected,0


### Data modification

In [29]:
# region has NaN values, let's see which NOC codes don't have a match in noc_regions
df[df.region.isnull()].NOC.unique()

array(['SGP', 'ROT', 'UNK', 'TUV'], dtype=object)

In [30]:
# let's get rid of the NaN region
df = df[~df.region.isnull()]

In [31]:
df.shape

(270746, 17)

In [34]:
# to create medal counts let's create gold_count, silver_count, bronze_count columns
df['gold_count'] = 0
df.loc[df.Medal=='Gold', 'gold_count'] = 1
df['silver_count'] = 0
df.loc[df.Medal=='Silver', 'silver_count'] = 1
df['bronze_count'] = 0
df.loc[df.Medal=='Bronze', 'bronze_count'] = 1
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes,gold_count,silver_count,bronze_count
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,0,0,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,0,0,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,0,0,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,1,0,0
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,0,0,0


### Data visualization

#### World map of medal winners

In [76]:
# load world geodata
world = gpd.read_file('./datasets/world_map/ne_110m_admin_0_countries.shp')
world = world[['ADMIN', 'ADM0_A3', 'geometry']].rename(columns={'ADMIN': 'country', 'ADM0_A3': 'country_code'})

# rename country names which don't match our olympic dataset
world.loc[world.country=='United States of America', 'country'] = 'USA'

In [77]:
world.head()

,country,country_code,geometry
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555..."
1,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...
2,Western Sahara,SAH,POLYGON ((-8.665589565454809 27.65642588959236...
3,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974..."
4,USA,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49...."


In [62]:
# make sure that NOC, region match SOVEREIGNT, SOV_A3
set(zip(world.ADMIN, world.ADM0_A3))

{('Afghanistan', 'AFG'),
 ('Albania', 'ALB'),
 ('Algeria', 'DZA'),
 ('Angola', 'AGO'),
 ('Antarctica', 'ATA'),
 ('Argentina', 'ARG'),
 ('Armenia', 'ARM'),
 ('Australia', 'AUS'),
 ('Austria', 'AUT'),
 ('Azerbaijan', 'AZE'),
 ('Bangladesh', 'BGD'),
 ('Belarus', 'BLR'),
 ('Belgium', 'BEL'),
 ('Belize', 'BLZ'),
 ('Benin', 'BEN'),
 ('Bhutan', 'BTN'),
 ('Bolivia', 'BOL'),
 ('Bosnia and Herzegovina', 'BIH'),
 ('Botswana', 'BWA'),
 ('Brazil', 'BRA'),
 ('Brunei', 'BRN'),
 ('Bulgaria', 'BGR'),
 ('Burkina Faso', 'BFA'),
 ('Burundi', 'BDI'),
 ('Cambodia', 'KHM'),
 ('Cameroon', 'CMR'),
 ('Canada', 'CAN'),
 ('Central African Republic', 'CAF'),
 ('Chad', 'TCD'),
 ('Chile', 'CHL'),
 ('China', 'CHN'),
 ('Colombia', 'COL'),
 ('Costa Rica', 'CRI'),
 ('Croatia', 'HRV'),
 ('Cuba', 'CUB'),
 ('Cyprus', 'CYP'),
 ('Czechia', 'CZE'),
 ('Democratic Republic of the Congo', 'COD'),
 ('Denmark', 'DNK'),
 ('Djibouti', 'DJI'),
 ('Dominican Republic', 'DOM'),
 ('East Timor', 'TLS'),
 ('Ecuador', 'ECU'),
 ('Egypt', 'EG

In [47]:
set(zip(regions.region, regions.NOC))

{('Afghanistan', 'AFG'),
 ('Albania', 'ALB'),
 ('Algeria', 'ALG'),
 ('American Samoa', 'ASA'),
 ('Andorra', 'AND'),
 ('Angola', 'ANG'),
 ('Antigua', 'ANT'),
 ('Argentina', 'ARG'),
 ('Armenia', 'ARM'),
 ('Aruba', 'ARU'),
 ('Australia', 'ANZ'),
 ('Australia', 'AUS'),
 ('Austria', 'AUT'),
 ('Azerbaijan', 'AZE'),
 ('Bahamas', 'BAH'),
 ('Bahrain', 'BRN'),
 ('Bangladesh', 'BAN'),
 ('Barbados', 'BAR'),
 ('Belarus', 'BLR'),
 ('Belgium', 'BEL'),
 ('Belize', 'BIZ'),
 ('Benin', 'BEN'),
 ('Bermuda', 'BER'),
 ('Bhutan', 'BHU'),
 ('Boliva', 'BOL'),
 ('Bosnia and Herzegovina', 'BIH'),
 ('Botswana', 'BOT'),
 ('Brazil', 'BRA'),
 ('Brunei', 'BRU'),
 ('Bulgaria', 'BUL'),
 ('Burkina Faso', 'BUR'),
 ('Burundi', 'BDI'),
 ('Cambodia', 'CAM'),
 ('Cameroon', 'CMR'),
 ('Canada', 'CAN'),
 ('Canada', 'NFL'),
 ('Cape Verde', 'CPV'),
 ('Cayman Islands', 'CAY'),
 ('Central African Republic', 'CAF'),
 ('Chad', 'CHA'),
 ('Chile', 'CHI'),
 ('China', 'CHN'),
 ('China', 'HKG'),
 ('Colombia', 'COL'),
 ('Comoros', 'COM'),


In [98]:
# prepare data to visualize: region, medal_count, gold_count, silver_count, bronze_count
medal_ranking = df[['region', 'gold_count', 'silver_count', 'bronze_count']].groupby(['region']).sum()
medal_ranking['total_count'] = medal_ranking['gold_count'] + medal_ranking['silver_count'] + medal_ranking['bronze_count']
medal_ranking.sort_values(by=['total_count'], ascending=False).head()

,gold_count,silver_count,bronze_count,total_count
region,,,,
USA,2638,1641,1358,5637
Russia,1599,1170,1178,3947
Germany,1301,1195,1260,3756
UK,678,739,651,2068
France,501,610,666,1777


In [99]:
# merge world geodata with medal ranking data
world_ranking = world.merge(medal_ranking, left_on='country', right_on='region')
world_ranking.shape

(159, 7)

In [100]:
medal_ranking.shape

(205, 4)

In [114]:
geo_source = GeoJSONDataSource(geojson=world_ranking.to_json())
color_mapper = LinearColorMapper(palette=Viridis256, low=medal_ranking.total_count.min(), high=medal_ranking.total_count.max())
addon_tools = [PanTool(), ZoomInTool(), ZoomOutTool()]

fig = figure(title='Olympic Medal Winner Map', toolbar_location='above', plot_width=1000, tools=addon_tools)
fig.patches(xs='xs', ys='ys', alpha=0.9, source=geo_source,
            color={'field': 'total_count', 'transform': color_mapper}
         )

hover = HoverTool(
    point_policy='follow_mouse',
    tooltips='<b>Country</b>: @country<br/><b>Total</b>: @total_count<br/><b>Gold</b>: @gold_count<br/><b>Silver</b>: @silver_count<br/><b>Bronze</b>: @bronze_count'
)
fig.add_tools(hover)
fig.xaxis.visible = False
fig.yaxis.visible = False
fig.grid.visible = False

color_bar = ColorBar(color_mapper=color_mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%.2f%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
fig.add_layout(color_bar, 'right')    
show(fig)